# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part III: Training our End Extraction Model

In this final section of the tutorial, we'll use the noisy training labels we generated in the last tutorial part to train our end extraction model.

For this tutorial, we will be training a simple - but fairly effective - logistic regression model.  More generally, however, Snorkel plugs in with many ML libraries including [TensorFlow](https://www.tensorflow.org/), making it easy to use almost any state-of-the-art model as the end extractor!

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Spouse` `Candidate` subclass, and load the test set:

In [2]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

We use the training marginals to train a discriminative model that classifies each `Candidate` as a true or false mention. We'll use a random hyperparameter search, evaluated on the development set labels, to find the best hyperparameters for our model. To run a hyperparameter search, we need labels for a development set. If they aren't already available, we can manually create labels using the Viewer.

In [3]:
from snorkel.annotations import load_marginals

train_marginals = load_marginals(session, split=0)

In [4]:
train_cands = session.query(Spouse).filter(Spouse.split == 0).order_by(Spouse.id).all()
dev_cands   = session.query(Spouse).filter(Spouse.split == 1).order_by(Spouse.id).all()
test_cands  = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()

In [5]:
from snorkel.annotations import load_gold_labels

L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, load_as_array=True, zero_one=True)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)

In [6]:
from snorkel.learning.disc_models.rnn import reRNN

train_kwargs = {
    'lr':         0.001,
    'dim':        100,
    'n_epochs':   10,
    'dropout':    0.5,
    'print_freq': 1,
    'max_sentence_length': 100
}

lstm = reRNN(seed=1701, n_threads=None)
lstm.train(train_cands, train_marginals, X_dev=dev_cands, Y_dev=L_gold_dev, **train_kwargs)

[reRNN] Training model
[reRNN] n_train=17142  #epochs=10  batch size=256
[reRNN] Epoch 0 (51.39s)	Average loss=0.585975	Dev F1=2.97
[reRNN] Epoch 1 (106.58s)	Average loss=0.543870	Dev F1=32.67
[reRNN] Epoch 2 (160.81s)	Average loss=0.538390	Dev F1=33.78
[reRNN] Epoch 3 (223.37s)	Average loss=0.537752	Dev F1=34.86
[reRNN] Epoch 4 (278.30s)	Average loss=0.537312	Dev F1=34.19
[reRNN] Epoch 5 (332.84s)	Average loss=0.537003	Dev F1=36.06
[reRNN] Epoch 6 (387.64s)	Average loss=0.537022	Dev F1=35.01
[reRNN] Epoch 7 (442.61s)	Average loss=0.536780	Dev F1=35.68
[reRNN] Model saved as <reRNN>
[reRNN] Epoch 8 (498.82s)	Average loss=0.536516	Dev F1=35.75
[reRNN] Model saved as <reRNN>
[reRNN] Epoch 9 (555.09s)	Average loss=0.536319	Dev F1=37.62
[reRNN] Model saved as <reRNN>
[reRNN] Training done (559.87s)
INFO:tensorflow:Restoring parameters from checkpoints/reRNN/reRNN-9
[reRNN] Loaded model <reRNN>


Now, we get the precision, recall, and F1 score from the discriminative model:

In [7]:
p, r, f1 = lstm.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.346, Recall: 0.549, F1 Score: 0.425


We can also get the candidates returned in sets (true positives, false positives, true negatives, false negatives) as well as a more detailed score report:

In [8]:
tp, fp, tn, fn = lstm.error_analysis(session, test_cands, L_gold_test)

Scores (Un-adjusted)
Pos. class accuracy: 0.549
Neg. class accuracy: 0.906
Precision            0.346
Recall               0.549
F1                   0.425
----------------------------------------
TP: 123 | FP: 232 | TN: 2241 | FN: 101



Note that if this is the final test set that you will be reporting final numbers on, to avoid biasing results you should not inspect results.  However you can run the model on your _development set_ and, as we did in the previous part with the generative labeling function model, inspect examples to do error analysis.

##### More importantly, you completed the introduction to Snorkel! Give yourself a pat on the back!